In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../notebook_format')
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

# some jupyter notebook magic to automatically reload the modules
# before executing the code
%load_ext autoreload
%autoreload 2

# Classification Decision Tree

Decision trees are extremely useful, very popular because of its interpretability and used a lot in the finance industry, particularly when looking at evaluating loan applications. So lets say, I want to buy a house and needs to take a loan from the bank. So the bank is going to look at my history record, like my credit, what has it been like in the past? How much money do I make? And other personal information about me, like my gender, age and so on. And the bank is then going to take that information and try to make a prediction as to whether loaning me money is a risky thing or not.

Below is an example decision tree of how the bank might arrive at its decision.

<img src='tree.png' width = 700 height = 700  >

So, the algorithm works by starting at the top of the tree (the root node) and it will go down one branch or one path of this decision tree and ask a series of questions, e.g. If somebody has a credit that's fair and he is applying for a short-term loan (less than three year in this case), then this is considered a risky application (the leaf node). 

We can then look at another person whose credit is poor, his income is high but he is still applying for a short term loan, and the decision tree will still consider it as a risky application. 

In general we're going to be given an input $x_i$. In this case it has values to each one of those input features that say credit poor, income high, and term five years longer term loan. And then we traverse down the path of the tree to make a prediction whether the loan would be safe. Our our task our course is to learn the tree that let's us make predictions about our data.

A core for learning a decision tree is a very simple recursive algorithm.

1. Start at the tree's root node (the one at the very top). It is called the parent node if the node is not at the top of the tree.
2. Select the best feature that splits the data for the current node. Here we'll be using binary classification tree, which simply splits the parent node into a constant of two child nodes.
3. For each split of the tree, we either make our predictions when the tree can't be further splitted or we return to step 1 and 2 to recursively split the tree deeper and deeper. 

There are a few points that we need to make more concrete. We have to decide how to pick the feature to split on. And then since this is a recursive algorithm here, we have to figure out when to stop the recursion, or simply when to not go and split another node in the tree.

If that explanation is still kind of blurry, try out this cool interactive visualization of how decision tree works. [A Visual Introduction to Machine Learning](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

## Splitting criteria for classification trees

The first question is what is the best feature to split on and how do we measure that?

Well, the simplest measurement is the **classification error rate: ** Fraction of observations in a region that don't belong to the most common class.

This simply means we'll predict all the samples with the majority class, and compute the classification error rate by: input number of the non-majority class divided by the total number of inputs at the current node.

Here's a concrete example, pretend that we are predicting whether someone buys an iPhone or an Android:

- At a particular node, there are **25 observations** (phone buyers), of whom **10 bought iPhones and 15 bought Androids**.
- Since the majority class is **Android**, that's our prediction for all 25 observations, thus the classification error rate is **10/25 = 40%**.

Our goal in making splits is to **reduce the classification error rate**. Let's try splitting on gender and suppose that it gave us the result of:

- **Males:** 2 iPhones and 12 Androids, thus the predicted class is Android
- **Females:** 8 iPhones and 3 Androids, thus the predicted class is iPhone
- Classification error rate after this split would be **5/25 = 20%**

Compare that with a split on age:

- **30 or younger:** 4 iPhones and 8 Androids, thus the predicted class is Android
- **31 or older:** 6 iPhones and 7 Androids, thus the predicted class is Android
- Classification error rate after this split would be **10/25 = 40%**

Note that here age is originally a continuous numerical feature. To use them in determining whether to use this feature as the splitting criteria we will often times group them into a small number of discrete categories. For instance, here we turned age into a binary class by indicating whether the person's age is 30 or younger or 31 or older.

The decision tree algorithm will try **every possible split across all features**, and choose the split that **reduces the error rate the most.** In this case, it will split on gender.

## Another Kind of Splitting Criteria

Another way to determine the best feature to split on is by choosing the one that maximizes the **Information Gain (IG)** at each split.

$$IG(D_{p}, a) = I(D_{p}) - \frac{N_{left}}{N_p} I(D_{left}) - \frac{N_{right}}{N_p} I(D_{right})$$

- $IG$: Information Gain.
- $a$: feature to perform the split.
- $N_p$: number of samples in the parent node.
- $N_{left}$: number of samples in the left child node.
- $N_{right}$: number of samples in the right child node.
- $I$: Some impurity measure that we'll look at below (common ones include gini index, entropy).
- $D_{p}$: training subset of the parent node.
- $D_{left}$: training subset of the left child node.
- $D_{right}$: training subset of the right child node.

**[Gini Index](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity)**

This is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Gini impurity can be computed by summing the probability $f_i$ of each item being chosen times the probability $1 − f_i$ of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category.

To compute Gini impurity for a set of m items, suppose $i ∈ {1, 2, ..., m}$, and let $f_i$ be the fraction of items labeled with value $i$ in the set.

$$ I = \sum _{i=1}^{m}f_{i}(1-f_{i})=\sum _{i=1}^{m}(f_{i}-{f_{i}}^{2})=\sum _{i=1}^{m}f_{i}-\sum _{i=1}^{m}{f_{i}}^{2}=1-\sum _{i=1}^{m}{f_{i}}^{2}$$

- The **maximum value** of the Gini index is 0.5, and occurs when the classes are perfectly balanced in a node.
- The **minimum value** of the Gini index is 0, and occurs when there is only one class represented in a node.
- A node with a lower Gini index is said to be more "pure".

### Example of Gini index

Calculate the Gini index before making a split:

$$1 - \left(\frac {iPhone} {Total}\right)^2 - \left(\frac {Android} {Total}\right)^2 = 1 - \left(\frac {10} {25}\right)^2 - \left(\frac {15} {25}\right)^2 = 0.48$$

Evaluating the split on **gender** using Gini index:

$$\text{Males: } 1 - \left(\frac {2} {14}\right)^2 - \left(\frac {12} {14}\right)^2 = 0.24$$
$$\text{Females: } 1 - \left(\frac {8} {11}\right)^2 - \left(\frac {3} {11}\right)^2 = 0.40$$
$$\text{Weighted Average: } 0.24 \left(\frac {14} {25}\right) + 0.40 \left(\frac {11} {25}\right) = 0.31$$
$$IG = 0.48 - 0.31 = 0.17$$

Evaluating the split on **age** using Gini index:

$$\text{30 or younger: } 1 - \left(\frac {4} {12}\right)^2 - \left(\frac {8} {12}\right)^2 = 0.44$$
$$\text{31 or older: } 1 - \left(\frac {6} {13}\right)^2 - \left(\frac {7} {13}\right)^2 = 0.50$$
$$\text{Weighted Average: } 0.44 \left(\frac {12} {25}\right) + 0.50 \left(\frac {13} {25}\right) = 0.47$$
$$IG = 0.48 - 0.47 = 0.01$$

Again, the decision tree algorithm will try every possible split, and will choose the split that reduces the Gini index (and thus increases the "node purity") the most. Or in other words, maximizes the information gain.

### Comparing Impurity Functions $I$

- Gini index is generally preferred because it will make splits that **increase node purity**, even if that split does not change the classification error rate.
- Node purity is important because we're interested in the **class proportions** in each region, since that's how we calculate the **predicted probability** of each class.
- scikit-learn's default splitting criteria for classification trees is Gini index.

Note: There is another common splitting criteria called **entropy**. Gini index and entropy typically yield very similar results in terms of the model's prediction accuracy, thus it is often not worth spending time on tuning different impurity criteria.

## When To Stop Recursing

There are some early stopping criteria that is commonly used to prevent the tree from overfitting.

**Maximum depth** The length of the longest path from a root node to a leaf node will not exceed this value. This is the most commonly tuned hyperparameter for tree-based method.

**Minimum node size:** This simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size.

In [14]:
# read in the data
titanic = pd.read_csv('titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
# encode female as 0 and male as 1
titanic['Sex'] = titanic.Sex.map({ 'female': 0, 'male': 1 })

# fill in the missing values for age with the median age
titanic['Age'].fillna( titanic['Age'].median(), inplace = True )

# one-hot encoded Embarked column
embarked_dummies = pd.get_dummies( titanic['Embarked'], prefix = 'Embarked' )

# concatenate the original DataFrame and the one-hot encoded DataFrame
titanic.drop( 'Embarked', axis = 1, inplace = True )
titanic = pd.concat([ titanic, embarked_dummies ], axis = 1 )
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0.0,0.0,1.0


In [16]:
# define X and y
features = ['Pclass', 'Sex', 'Age', 'Embarked_C', 'Embarked_Q', 'Embarked_S'] 
X = titanic[features]
y = titanic['Survived']

In [17]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 1 )
print(X_train.shape)
print(X_test.shape)

(712, 6)
(179, 6)


In [20]:
# accuracy score of one decision tree classifier
tree = DecisionTreeClassifier( max_depth = 6 )
tree.fit( X_train, y_train )
y_pred = tree.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

0.77094972067039103

In [25]:
rf = RandomForestClassifier( n_estimators = 30, max_depth = 6, max_features = 'sqrt' )
rf.fit( X_train, y_train )
y_pred = rf.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

0.76536312849162014

In [24]:
?RandomForestClassifier

## Decision Tree Implementation

Link to download the data from [dropbox](https://www.dropbox.com/s/feo2z4wg8khcuct/lending-club-data.csv?dl=0). 

We will build a classification model to predict whether or not a loan provided by [LendingClub](https://www.lendingclub.com/) is likely to default. The dataset has feature columns that have to do with grade of the loan, annual income, home ownership status, etc. We can take a look at the distribution of the `home_ownership`, which describes whether the loanee is mortaging, renting, or owns a home.

In [26]:
loans = pd.read_csv('lending-club-data.csv')

# a small percentage of the loanees own a home
print( loans['home_ownership'].value_counts() )
print()
loans.head()

MORTGAGE    59240
RENT        53245
OWN          9943
OTHER         179
Name: home_ownership, dtype: int64



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column **1** means a risky (bad) loan **0** means a safe loan. Here, we'll reassign the target to be:

- **1** as a safe loan.
- **0** as a risky (bad) loan. 

We put this in a new column called `safe_loans`. This step isn't necessary, it's more of a personal preference for the interpretation of the outcome.

In [27]:
loans['safe_loans'] = loans['bad_loans'].apply( lambda x: 1 if x == 0 else 0 )
loans.drop( 'bad_loans', axis = 1, inplace = True )

# it looks like most of these loans are safe loans (thankfully)
# But this does make our problem of identifying risky loans challenging
loans['safe_loans'].value_counts()

1    99457
0    23150
Name: safe_loans, dtype: int64

For now, we will just be using a subset of 4 categorical features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

We'll also convert these categorical features to a binary representation using one-hot encoding. For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is takes the value of either `own`, `mortgage` or `rent`. Now, if a data point has the feature: 

```
{   'home_ownership': 'RENT'   }
```

we want to turn this into three features: 

```
{ 
    'home_ownership = OWN'      : 0, 
    'home_ownership = MORTGAGE' : 0, 
    'home_ownership = RENT'     : 1
}
```
Where the 1 denotes that the data point contains this feature, and 0 otherwise.

In [28]:
# extract the subset of feature columns and the target column
features = [ 'grade', 'term', 'home_ownership', 'emp_length' ]
target = 'safe_loans'
loans  = loans[ features + [target] ]

# one-hot encoding
loans = pd.get_dummies(loans)
loans.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_n/a
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# obtain the features after one-hot encoded,
# the first column is the target
features = loans.columns[1:]
X = loans[features].values
y = loans[target].values

# train / test split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 1 )
print(X_train.shape)
print(X_test.shape)

(98085, 25)
(24522, 25)


In [30]:
# accuracy score of one decision tree classifier
tree = DecisionTreeClassifier( max_depth = 6 )
tree.fit( X_train, y_train )
y_pred = tree.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

0.80976266209933934

In [31]:
rf = RandomForestClassifier( n_estimators = 50, max_depth = 6, max_features = 'sqrt' )
rf.fit( X_train, y_train )
y_pred = rf.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

0.80980344180735664

### Getting Started

In [73]:
def best_splitting_feature( X, y ):
    """
    Loop through the possible feature to consider splitting on each on them
    and return the best feature we found. Where best is determined by the 
    maximum information gain that uses gini index as the impurity score
    
    Parameters
    ----------
    X, y : np-array
        data and its target value
        
    features: list
        candidates (columns) to be considered for splits
        
    target: string
        name of the target/label column
        
    Returns
    -------
    best_feature: int
        column index of the best feature we've found
    """
    
    # keep track of the best score for the splitting criteria and its corresponding feature
    # we'll simply intialize the best score to be some small number
    best_feature = None
    best_score = -np.inf
    
    # compute the impurity score, here in gini index for the current data points
    original_score = compute_node_score(y)
    data_num, feature_num = X.shape

    for f in range(feature_num):
              
        # Since we've one hot encoded the data points,
        # the left split will have all data points where the feature value is 0 and
        # the right split will have all data points where the feature value is 1
        split = X[ :, f ]
        y_left  = y[ split == 0 ]
        y_right = y[ split == 1 ] 
            
        # compute the impurity score for the left and right split
        left_score  = compute_node_score( y = y_left  )
        right_score = compute_node_score( y = y_right )
        
        # compute the information gain     
        left_data_num  = y_left.shape[0]
        right_data_num = y_right.shape[0]        
        split_score = ( ( right_data_num / data_num ) * right_score + 
                        ( left_data_num / data_num ) * left_score )
        score = original_score - split_score

        if score > best_score:
            best_score = score
            best_feature = f
    
    return best_feature

In [57]:
def compute_fraction(y):
    data_num = y.shape[0]
    labels_num = np.bincount(y)
    fraction = labels_num / data_num
    return fraction

In [75]:
def compute_node_score(y):
    # print(y)
    """compute the impurity score, the gini index of the current node"""
    fraction = compute_fraction(y)
    gini_index = 1 - np.sum( fraction ** 2 )
    return gini_index

In [59]:
def create_leaf(y):
    
    # add the predicted probability of each label
    # to the dictionary, which is simply the fraction
    fraction = compute_fraction(y)
    leaf = { 
        'is_leaf'   : True,  
        'prediction': fraction
    }
    return leaf 

In [60]:
def fit( X, y ):
    feature_num = X.shape[1]
    features = list( range(feature_num) )    
    node = create_decision_tree( X = X, y = y, features = features )   
    return node

In [74]:
def create_decision_tree( X, y, features, current_depth = 0 ):
    
    # make a copy of the features
    remaining_features = features[:] 
       
    # define stopping condition, 
    # if it is reached, make the current node a leaf node
    
    # if the number of data points is less than or equal to the minimum size
    if X.shape[0] <= min_node_size:
        return create_leaf(y)
    
    # if there are no remaining features to consider splitting on for one branch
    if not remaining_features:
        return create_leaf(y)    
    
    # limit the tree depth
    if current_depth >= max_depth:
        return create_leaf(y)

    # find the best splitting feature and split on it
    splitting_feature = best_splitting_feature( X, y )
    split = X[ :, splitting_feature ]
    X_left, X_right = X[ split == 0 ], X[ split == 1 ]
    y_left, y_right = y[ split == 0 ], y[ split == 1 ]
        
    # recursively build the left and right sub-trees
    remaining_features = list( set(remaining_features) - set([splitting_feature]) )
    
    left_tree = create_decision_tree( 
        X = X_left,
        y = y_left,
        features = remaining_features,     
        current_depth = current_depth + 1, 
    )        
    right_tree = create_decision_tree( 
        X = X_right,
        y = y_right,
        features = remaining_features,        
        current_depth = current_depth + 1
    )
    
    node = { 
        'is_leaf'          : False, 
        'left'             : left_tree,
        'right'            : right_tree,
        'splitting_feature': splitting_feature
    }
    return node

In [108]:
def classify( x, tree ):   

    if tree['is_leaf']:
        return tree['prediction'] 
    else:
        # split on feature, splitted feature value of 0 is grown on the left-subtree
        split_feature_value = x[ tree['splitting_feature'] ]
        if split_feature_value == 0:
            return classify( x, tree['left'] )
        else:
            return classify( x, tree['right'] )

In [76]:
min_node_size = 10
max_depth = 6

tree_model = fit( X_train, y_train )

In [113]:
prediction = np.apply_along_axis( func1d = classify, axis = 1, arr = X_test, tree = tree_model )

In [115]:
prediction

array([[ 0.27610759,  0.72389241],
       [ 0.07545563,  0.92454437],
       [ 0.15343811,  0.84656189],
       ..., 
       [ 0.06092199,  0.93907801],
       [ 0.27610759,  0.72389241],
       [ 0.12392451,  0.87607549]])

**Advantages of decision trees:**

- Can be displayed graphically, thus making it highly interpretable.
- Features don't need scaling or normalization.
- Tends to ignore irrelevant features.
- Non-parametric (will outperform linear models if relationship between features and response is highly non-linear).

# Ensemble Methods

Ensembling is a very popular method for improving the predictive performance of machine learning models. Let's pretend that instead of building a single model to solve a binary classification problem, you created **five independent models**, and each model was correct about 70% of the time. If you combined these models into an "ensemble" and used their majority vote as a prediction, how often would the ensemble be correct?

In [3]:
# generate 1000 random numbers (between 0 and 1) for each model, representing 1000 observations
np.random.seed(1234)
mod1 = np.random.rand(1000)
mod2 = np.random.rand(1000)
mod3 = np.random.rand(1000)
mod4 = np.random.rand(1000)
mod5 = np.random.rand(1000)

# each model independently predicts 1 (the "correct response") if random number was at least 0.3
preds1 = np.where(mod1 > 0.3, 1, 0)
preds2 = np.where(mod2 > 0.3, 1, 0)
preds3 = np.where(mod3 > 0.3, 1, 0)
preds4 = np.where(mod4 > 0.3, 1, 0)
preds5 = np.where(mod5 > 0.3, 1, 0)

# print the first 20 predictions from each model
print(preds1[:20])
print(preds2[:20])
print(preds3[:20])
print(preds4[:20])
print(preds5[:20])

# how accurate was each individual model?
print(preds1.mean())
print(preds2.mean())
print(preds3.mean())
print(preds4.mean())
print(preds5.mean()) 

[0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1]
[1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0 0]
[1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1]
[1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 0]
[0 0 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1]
0.713
0.665
0.717
0.712
0.687


In [4]:
# average the predictions, and then round to 0 or 1
# you can also do a weighted average, as long as the weight adds up to 1 
ensemble_preds = np.round((preds1 + preds2 + preds3 + preds4 + preds5) / 5).astype(int)

# print the ensemble's first 20 predictions
print(ensemble_preds[:20])

# how accurate was the ensemble?
print(ensemble_preds.mean())

[1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1]
0.841


**Ensemble learning (or "ensembling")** is the process of combining several predictive models in order to produce a combined model that is more accurate than any individual model.

- **Regression:** take the average of the predictions
- **Classification:** take a vote and use the most common prediction, or take the average of the predicted probabilities

The big idea: If you have a collection of individually imperfect, but different models (by different, it might mean that it uses different hyperparameters, features or class of algorithms), the "one-off" mistakes made by each model are probably not going to be made by the rest of the models, and thus the mistakes will be discarded when averaging the models.

There are two basic methods for ensembling:

- Manually ensemble your individual models (e.g. the weighted average method above).
- Use a model that ensembles for you, which we'll see below.


## Randomforest

The primary weakness of **decision trees** is that they usually suffer from high variance and don't tend to have the best predictive accuracy. This means that if we split the training data into two parts at random, and fit a decision tree to both halves, the results that we get could be quite different. Thus in a **randomforest**, we will grow multiple INDEPENDENT decision trees as opposed to a single tree. Then, to classify a new object based on attributes, the forest chooses the classification having the most votes or the weighted probability (over all the trees in the forest) and in case of regression, it takes the average of outputs by different trees.

In randomforest, each tree is trained as follows:

- Assume number of observations in the training set is $N$. Then, sample of these $N$ cases is taken at random but with replacement (i.e. bagging or bootstrap aggregation). This sample will be the training set for growing the tree. Notice here, each tree in randomforest is built on a different bootstrap data set, making it independent of all the other trees.
- If there are $M$ total input variables (features), a number of $ m < M$ features is chosen randomly to become our split candidates from the full set of $M$ total features (without replacement this time). The best split on these $m$ is then used to split the node. The value of $m$ is held constant while growing each tree. In the regression context, it is suggested to set $m$ to be one-third of $M$. As for classification, square root of $M$ is usually the default. The point of doing this is that, suppose there is one very strong feature in the data set. When training the trees, most of the trees will use that feature as the top split, resulting in an ensemble of similar trees that are highly correlated. By randomly leaving out candidate features from each split, Random Forests "decorrelates" the trees, such that the averaging process can reduce the variance of the resulting model.
- Each tree is grown to the largest extent possible (you can specify the max_depth parameter to control how large) and there is no pruning.

In the end, we'll predict new data by aggregating the predictions of the number trees (e.g., majority votes for classification, average for regression)

If this makes absolutely no sense to you, try this more intuitive explanation from [Quora](https://www.quora.com/How-does-randomization-in-a-random-forest-work?redirected_qid=212859).

**Toy Implementation:** Built on top of scikit learn's decision tree.

The example [digit dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) is made up of 1797 8x8 images. Each image, is of a hand-written digit. In order to utilize an 8x8 figure like this, we’d have to first transform it into a feature vector with length 64. There're a total of 1797 examples and 10 classes.

In [17]:
from sklearn.datasets import load_digits

In [6]:
# load the data and split it into training and testing
digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 1 )
print(X_train.shape)
print(X_test.shape)

(1437, 64)
(360, 64)


In [99]:
# accuracy score of one decision tree classifier
tree = DecisionTreeClassifier()
tree.fit( X_train, y_train )
y_pred = tree.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

0.80906940706304542

Side note, the best thing about decision trees is that can be displayed graphically, and are easily interpreted even by a non-expert. Thus the following section provides a code for visualizing the tree. Note that you'll need to have [graphviz](http://www.graphviz.org/) installed. Or this [link](http://macappstore.org/graphviz-2/) if you're a mac user.

```python

from sklearn.tree import export_graphviz
import graphviz

# export it as .dot file, other common parameters include
# `feature_names` 
# `fill` (for filling the node with different output type)
# `class_names`
# `rounded` (boolean to round the score on each node)
export_graphviz( tree, out_file = "tree.dot" )

# read it in and visualize it
with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

# or if you wish to convert the .dot file into other formats
import os
os.system("dot -Tpng tree.dot -o tree.jpeg")
```

In [21]:
class RandomForest(object):
    """
    vanilla classification random forest
    
    the two core features for random forest
    is the n_estimators, the number of trees that you're going built
    and the max_features, the number of features that you allow
    when deciding which feature to split on, you can still specify
    all the other parameters for a decision tree like math_depth
    or min_sample_split, it is just not used here as that is more
    related to a single decision tree
    """   
    def __init__( self, n_estimators, max_features ):
        self.n_estimators = n_estimators
        self.max_features = max_features
        
    
    def fit( self, X, y ):       
        # draw the bootstrap samples
        n_samples = y.shape[0]
           
        # for each number of base-tree models to build
        # 1. draw bootstrap samples from the original data
        # 2. train a tree model on that bootstrap sample,
        #    during training, randomly select a number of features to split on each node
        self.forest = []
        for _ in range(self.n_estimators):
            bootstrap = np.random.choice( n_samples, size = n_samples, replace = True )
            X_boostrap = X[bootstrap]
            y_boostrap = y[bootstrap]
            
            # using scikit learn's decision tree as the base tree
            tree = DecisionTreeClassifier( max_features = self.max_features )
            tree.fit( X_boostrap, y_boostrap )
            self.forest.append(tree)
            
    
    def predict( self, X ):        
        # obtain the prediction for each base-tree model
        # then use majority vote to decide the predicted class
        # for each observation
        n_samples = X.shape[0]
        tree_pred = np.zeros( ( n_samples, self.n_estimators ), dtype = np.int )       
        for j in range(self.n_estimators):
            tree = self.forest[j]
            tree_pred[ :, j ] = tree.predict(X)

        y_pred = np.zeros(n_samples)
        for i in range(n_samples):
            y_pred[i] = np.argmax( np.bincount( tree_pred[ i, : ] ) )
        
        return y_pred

In [22]:
# accuracy score of random forest classifier using 30 decision trees
rf = RandomForest( n_estimators = 30, max_features = 'sqrt' )
rf.fit( X_train, y_train )
y_pred = rf.predict(X_test)
metrics.accuracy_score( y_true = y_test, y_pred = y_pred )

IndexError: indices are out-of-bounds

## Gradient Boosting (Regression)

**Gradient boosting machines** is a method stemmed from boosting. The basic principles of the algorithm are as follows: given a loss function (e.g., squared error for regression) and a weak learner (e.g., regression trees), the algorithm seeks to find an additive model that minimizes the loss function. The algorithm is typically initialized with the best guess of the response (e.g., the mean of the response in regression). The gradient (e.g., residual) is calculated, and a model is then fit to the residuals to minimize the loss function. The current model is added to the previous model, and the procedure continues for a user-specified number of iterations.

If squared error is used as the loss function, then a simple psuedocode for the algorithm is listed below.

- Compute the average response, $\bar{y}$, and use this as the initial predicted value for each sample
- for $b = 1, 2, ..., B$: ($B$ is the number of boosted trees, or simply think of it as iterations)
    - Compute the residual, the difference between the observed value and the current predicted value, for each sample
    Fit a regression tree of depth, D, using the residuals as the response
    Predict each sample using the regression tree fit in the previous step
    Update the predicted value of each sample by adding the previous iteration's predicted value to the predicted value 
    generated in the previous step

Clearly, this version of boosting has similarities to random forests: the final prediction is based on an ensemble of models, and trees are used as the base learner. However, the way the ensembles are constructed differs substantially between each model. In random forests, all trees are created independently, which means that each tree contributes equally to the final model. The trees in boosting, however, are dependent on past trees and contribute unequally to the final model. Despite these differences, both random forests and boosting offer competitive predictive performance.

In [14]:
# Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
y = boston["target"]
names = boston["feature_names"]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, random_state = 1 )
print(X_train.shape)
print(X_test.shape)

(404, 13)
(102, 13)


In [19]:
decision_tree = DecisionTreeRegressor()
decision_tree.fit( X_train, y_train )   
y_pred = decision_tree.predict(X_test)
metrics.mean_squared_error( y_true = y_test, y_pred = y_pred )

17.824803921568627

In [26]:
# number of trees to train,
# or in xgboost, the num_boost_round
n_estimators = 30

# learning rate
eta = 0.1

In [27]:


trees = []

residuals = y_train.copy()
for i in range(n_estimators):
    tree = DecisionTreeRegressor()
    tree.fit( X_train, residuals )   
    predictions = tree.predict(X_train)
    
    trees.append(tree)
    
    residuals -= eta * predictions

y_pred = np.zeros(X_test.shape[0])
for tree in trees:
    y_pred += eta * tree.predict(X_test)

metrics.mean_squared_error( y_true = y_test, y_pred = y_pred )

10.12239174738148

## Feature Importance

Typically randomforest results in improved accuracy over prediction using a single tree. Unfortunately, it can be difficult to interpret the resulting model since we're basically training a whole bunch of decision trees and averaging their "decisions". We can still, however, obtain an overall summary of the importance of each predictor using **mean decrease impurity** or **mean decrease accuracy**.

We'll start with **mean decrease impurity**. Recall that training a decision tree, every node in the tree is a condition on a single feature, designed to split the dataset into two so that similar response values end up in the same set. During this process, the node is chosen based on the (local) optimal decrease in impurity, which is typically either Gini impurity or information gain/entropy for classification trees and RSS (residual sum of squares )for regression trees. Thus when training a tree, it can be recorded how much each feature decreases the impurity.

Now that we've have this notion, we can extend the idea to randomforest. Since it is basically consists of a number of decision trees, the impurity decrease from each feature can simply be averaged over all the trees and the features are then ranked according to this measure (A large value indicates an important predictor).

In [11]:
rf = RandomForestRegressor( n_estimators = 30 )
rf.fit( X_train, y_train )
imp = rf.feature_importances_

indices = np.argsort(imp)
imp_df = pd.DataFrame({ 'feature': names[indices], 'imp': imp[indices] })
imp_df.sort_values( 'imp', ascending = False, inplace = True )
imp_df.head()

(404, 13)
(102, 13)


,feature,imp
12,LSTAT,0.483755
11,RM,0.304860
10,DIS,0.058533
9,CRIM,0.046649
8,NOX,0.030843


There are a few gotchas to keep in mind when using the impurity based ranking. 

- Firstly, feature selection based on impurity reduction is biased towards preferring variables with more categories. 
- Secondly, when the dataset has two (or more) correlated features, then from the point of view of the model, any of these correlated features can be used as the predictor, with no concrete preference of one over the others. But once one of them is used, the importance of others is significantly reduced since effectively the impurity they can remove is already removed by the first feature. As a consequence, they will have a lower reported importance. This is not an issue when we want to use feature selection to reduce overfitting, since it makes sense to remove features that are mostly duplicated by other features. But when interpreting the data, it can lead to the incorrect conclusion that one of the variables is a strong predictor while the others in the same group are unimportant, while actually they are very close in terms of their relationship with the response variable. We'll show this notion with a toy example below.

In the following example, we have three correlated variables X0,X1,X2, with the output variable simply being the sum of the three features:

In [10]:
size = 10000
np.random.seed(seed = 10)
X_seed = np.random.normal(0, 1, size)
X0 = X_seed + np.random.normal(0, .1, size)
X1 = X_seed + np.random.normal(0, .1, size)
X2 = X_seed + np.random.normal(0, .1, size)
X = np.array([X0, X1, X2]).T
Y = X0 + X1 + X2
  
# 20 trees, at each split, only two of the three features are considered
rf_reg = RandomForestRegressor(n_estimators = 20, max_features = 2 )
rf_reg.fit( X, Y )
print( "Scores for X0, X1, X2:", rf_reg.feature_importances_ )

Scores for X0, X1, X2: [ 0.27227966  0.54196882  0.18575152]


When we compute the feature importances, we see that X1 is computed to have over 10x higher importance than X2, while their "true" importance should be very similar. One thing to point out though is that the difficulty of interpreting the importance/ranking of correlated variables is not random forest specific, but applies to most model based feature selection methods.

**Mean decrease accuracy**

Another feature selection method is to directly measure the impact of each feature on accuracy of the model. The general idea is to permute the values of each feature and measure how much the permutation decreases the accuracy of the model. Clearly, for unimportant variables, the permutation should have little to no effect on model accuracy, while permuting important variables should significantly.

In [12]:
def feature_importance( rf, X, y ):
    
    y_pred = rf.predict(X)
    original_score = metrics.r2_score( y_true = y, y_pred = y_pred )
    
    imp = []
    for j in range(X.shape[1]):
        
        # copy the original data and shuffle one of the feauture's row order
        X_copy = X.copy()
        np.random.shuffle( X_copy[ :, j ] )
        
        y_pred = rf.predict(X_copy)
        shuffled_score = metrics.r2_score( y_true = y, y_pred = y_pred )
        diff_score = ( original_score - shuffled_score ) / original_score       
        imp.append(diff_score)
        
    return imp

In [13]:
imp = feature_importance( rf, X = X_test, y = y_test )
imp

[0.036421945019896089,
 -0.0013419756313426251,
 -0.00065772953953957948,
 0.00030352150544105734,
 0.030569578679355832,
 0.54667509315923135,
 0.0059797550370029604,
 0.059004156657436767,
 -0.0024793839798306124,
 0.0056676481991816364,
 0.055628170042757344,
 0.011605390220211146,
 0.94217739292137237]

In [129]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
from sklearn.datasets import load_boston
# Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]
 
rf = RandomForestRegressor()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 100, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
print( "Features sorted by their score:" )
print(sorted([(round(np.mean(score), 4), feat) for
              feat, score in scores.items()], reverse=True) )

Features sorted by their score:
[(0.75180000000000002, 'LSTAT'), (0.52600000000000002, 'RM'), (0.088200000000000001, 'DIS'), (0.042599999999999999, 'NOX'), (0.042000000000000003, 'CRIM'), (0.019099999999999999, 'PTRATIO'), (0.0167, 'TAX'), (0.010999999999999999, 'AGE'), (0.0055999999999999999, 'B'), (0.0041999999999999997, 'INDUS'), (0.0030000000000000001, 'RAD'), (0.00029999999999999997, 'CHAS'), (0.0, 'ZN')]


In this example LSTAT and RM are two features that strongly impact model performance: permuting them decreases model performance by ~73% and ~57% respectively. Keep in mind though that these measurements are made only after the model has been trained (and is depending) on all of these features. This doesn’t mean that if we train the model without one these feature, the model performance will drop by that amount, since other, correlated features can be used instead.

## Reference

- Decision Tree
    - [Coursersa's Course: Washington Classification](https://www.coursera.org/learn/ml-classification)
    - [A Visual Introduction to Machine Learning](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)
    - [Cheatsheet for Decision Tree Classification](http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/decision_trees/decision-tree-cheatsheet.ipynb)
    - [General Assembly's Notebook On Decision tree](http://nbviewer.jupyter.org/github/justmarkham/DAT8/blob/master/notebooks/17_decision_trees.ipynb)
- Randomforest
    - [Analyticsvidhya: randomforest](http://www.analyticsvidhya.com/blog/2015/09/random-forest-algorithm-multiple-challenges/)
    - [Quora: How does randomization in a random forest work?](https://www.quora.com/How-does-randomization-in-a-random-forest-work?redirected_qid=212859)
    - [General Assembly's Notebook On Ensembling](http://nbviewer.jupyter.org/github/justmarkham/DAT8/blob/master/notebooks/18_ensembling.ipynb)
    - [Blog: Selecting good features with random forests](http://blog.datadive.net/selecting-good-features-part-iii-random-forests/)

https://www.quora.com/What-is-an-intuitive-explanation-of-Gradient-Boosting

http://stats.stackexchange.com/questions/135378/how-are-individual-trees-added-together-in-boosted-regression-tree

https://www.youtube.com/watch?v=sRktKszFmSk

http://nbviewer.jupyter.org/github/leig/Applied-Predictive-Modeling-with-Python/blob/master/notebooks/Chapter%208.ipynb

http://localhost:8888/notebooks/machine-learning/trees/Chapter%208.ipynb

http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf

### variable importance

http://stats.stackexchange.com/questions/162162/relative-variable-importance-for-boosting
    
http://papers.nips.cc/paper/4928-understanding-variable-importances-in-forests-of-randomized-trees.pdf



http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#varimp



### trees

http://localhost:8888/notebooks/washington-machine-learning-specialization/course-3/module-6-decision-tree-practical-assignment.ipynb

